# Utils

In [ ]:
#|default_exp utils

In [ ]:
#| export
import numpy as np
import pandas as pd
from fastcore.all import *
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from tsai.utils import yaml2dict
from tsai.data.external import download_data
from collections import Counter
from itertools import combinations, chain
from tqdm import tqdm

config = yaml2dict('../dev_nbs/config/solfsmy.yaml', attrdict=True)

In [ ]:
#| export
def plot_forecast_2(X_true, y_true, y_pred, dtms=None, sel_vars=None, idx=None, figsize=(8, 4), n_samples=1):
    #TODO: add support for dynamic x axis interval in set_major_locator
    import matplotlib.pyplot as plt
    import matplotlib.patches as mpatches

    def _plot_forecast(X_true, y_true, y_pred, sel_var=None, idx=None, figsize=(8, 4)):
        if idx is None:
            idx = np.random.randint(0, len(X_true))
        if sel_var is None:
            title = f'sample: {idx}'
        else:
            title = f'sample: {idx} sel_var: {sel_var}'
        if sel_var is None: sel_var = slice(None)

        pred = np.concatenate([X_true[idx, sel_var], y_true[idx, sel_var]], -1)
        pred[..., :X_true.shape[-1]] = np.nan

        true = np.concatenate([X_true[idx, sel_var], y_pred[idx, sel_var]], -1)
        true_hist = true.copy()
        true_fut = true.copy()

        true_hist[..., X_true.shape[-1]:] = np.nan
        true_fut[..., :X_true.shape[-1]] = np.nan
                
        plt.figure(figsize=figsize)
        if dtms is not None:
            #dtms_plot = pd.to_datetime(dtms[idx])
            dtms_plot = mdates.date2num(dtms[idx])
            plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
            plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=10))
            plt.xlim(min(dtms_plot), max(dtms_plot))
            plt.plot(dtms_plot, pred.T, color='orange', lw=1, linestyle='--')
            plt.plot(dtms_plot, true_hist.T, color='purple', lw=1)
            plt.plot(dtms_plot, true_fut.T, color='purple', lw=1, linestyle='--')
            plt.axvline(dtms_plot[X_true.shape[-1]-1], color='gray', lw=.5, linestyle='--')
        else:
            plt.xlim(0, X_true.shape[-1] + y_true.shape[-1])
            plt.plot(pred.T, color='orange', lw=1, linestyle='--')
            plt.plot(true_hist.T, color='purple', lw=1)
            plt.plot(true_fut.T, color='purple', lw=1, linestyle='--')
            plt.axvline(X_true.shape[-1] - 1, color='gray', lw=.5, linestyle='--')

        plt.title(title)
        pred_patch = mpatches.Patch(color='orange', label='pred')
        true_patch = mpatches.Patch(color='purple', label='true')
        plt.legend(handles=[true_patch, pred_patch], loc='best')
        plt.show()
      
    assert X_true.shape[:-1] == y_true.shape[:-1] == y_pred.shape[:-1]
    assert y_true.shape[-1] == y_pred.shape[-1]
    
    if idx is not None:
        idx = listify(idx)
        n_samples = len(idx)
        iterator = idx
    else:
        iterator = random_randint(len(X_true), size=n_samples)
    
    if sel_vars is None:
        for idx in iterator:
            _plot_forecast(X_true, y_true, y_pred, sel_var=None, idx=idx, figsize=figsize)
    else:
        for idx in iterator:
            if sel_vars is True:
                sel_vars = np.arange(y_true.shape[1])
            else:
                sel_vars = listify(sel_vars)
            for sel_var in sel_vars:
                _plot_forecast(X_true, y_true, y_pred, sel_var=sel_var, idx=idx, figsize=figsize)


In [ ]:
# Test
n_samples = 100
n_vars = 10
lookback = 5
horizon = 5
X_true = np.random.randn(n_samples, n_vars, lookback)
y_true = np.random.randn(n_samples, n_vars, horizon)
y_pred = np.random.randn(n_samples, n_vars, horizon)
# dtms must be a 2D numpy array of datetime objects with the shape 
# (n_samples x (lookback + horizon))
dtms = np.array([pd.date_range('2020-01-01', 
                               periods=lookback + horizon, 
                               freq='D') for i in range(n_samples)])
plot_forecast_2(X_true, y_true, y_pred, dtms=dtms, sel_vars=0, idx=0, 
                figsize=(8, 4), n_samples=1)

In [ ]:
#| export

def plot_solar_algorithm_performance(df, var, figsize=(10, 10), ylims_mean=None, 
                                     ylims_std=None):
    # Plot a grid where each row is a solar activity level, and each column
    # is a error type (sfu or percent). Each cell is the result of calling the 
    # function plot_fe.
    # Input:
    # df: dataframe with the results of the forecasting experiment, with the columns
    #     variable, condition, horizon, mean_sfu, std_sfu, mean_percent, std_percent
    # var: variable to plot (F10, S10, M10, Y10)
    # figsize: figure size
    # ylims_mean: List with the y limits of the mean for each error type: 
    #   [(percent[0], percent[1]), (sfu[0], sfu[1])]
    # ylims_std: List with the y limits of the standard deviation for each error type:
    #   [(percent[0], percent[1]), (sfu[0], sfu[1])]
    # Output:
    # None, but it plots the grid
    sals = ['low', 'moderate', 'elevated', 'high']
    fig, axs = plt.subplots(len(sals), 2, figsize=figsize)
    fig.suptitle(f"Forecast error for {var}")
    for sal_idx, sal in enumerate(sals):
        for idx, err_type in enumerate(['percent', 'sfu']):
            df_var = df[(df['variable'] == var) & (df['condition'] == sal)]
            # Minimum and maximum values across column
            min_val_mean = df[f'mean_{err_type}'].min() if ylims_mean is None else ylims_mean[idx][0]
            max_val_mean = df[f'mean_{err_type}'].max() if ylims_mean is None else ylims_mean[idx][1]
            min_val_std = df[f'std_{err_type}'].min() if ylims_std is None else ylims_std[idx][0]
            max_val_std = df[f'std_{err_type}'].max() if ylims_std is None else ylims_std[idx][1]

            mean_fe = df_var[f'mean_{err_type}'].values
            std_fe = df_var[f'std_{err_type}'].values
            ax = axs[sal_idx, idx]
            ax.plot(mean_fe, color='#000000')
            ax.set_xlabel('Days from Epoch')
            ax.set_ylabel(f'Mean [{err_type}]', color='#000000')
            ax.tick_params(axis='y', labelcolor='#000000')
            ax.set_xticks(range(len(mean_fe)))
            ax.set_xticklabels(range(1, len(mean_fe)+1))
            ax.set_ylim(min_val_mean, max_val_mean)
            ax2 = ax.twinx()
            ax2.plot(std_fe, color='tab:red')
            ax2.set_ylabel(f'STD [{err_type}]', color='tab:red')
            ax2.set_ylim(min_val_std, max_val_std)
            ax2.tick_params(axis='y', labelcolor='tab:red')
            # ax2.set_xticks(range(len(std_fe)))
            # ax2.set_xticklabels(range(1, len(std_fe)+1))
            n_samples = df_var['n_samples'].values[0] 
            ax.set_title(f'{sal}\n{n_samples} forecasts')
            # Draw a grid in the background
            ax.grid(True, which='both', axis='both', color='lightgrey',
                    linestyle='-', linewidth=0.5)
            fig.tight_layout()

In [ ]:
# Test 
df = pd.DataFrame({'variable': ['F10', 'F10', 'F10', 'F10', 'S10', 'S10', 'S10', 'S10'],
                   'condition': ['low', 'moderate', 'elevated', 'high', 'low', 'moderate', 'elevated', 'high'],
                   'horizon': [1, 1, 1, 1, 1, 1, 1, 1],
                   'n_samples': [100, 100, 100, 100, 100, 100, 100, 100],
                   'mean_percent': [10, 20, 30, 40, 10, 20, 30, 40],
                   'std_percent': [1, 2, 3, 4, 1, 2, 3, 4],
                   'mean_sfu': [100, 200, 300, 400, 100, 200, 300, 400],
                   'std_sfu': [10, 20, 30, 40, 10, 20, 30, 40]})
plot_solar_algorithm_performance(df, 'F10', figsize=(10, 10))

In [ ]:
#|export
def bold_best(X, X_ref, higher_better=False, bold_ref_too=False, 
              bold_equal=False, use_abs=False):
    """
        Returns X with the best values in bold, with respect to X_ref, position by
        position, i.e., if X[0] is better than X_ref[0] it will be bolded. 
        Input:
            X: 1D numpy array
            X_ref: 1D numpy array
            higher_better: If True, then the best values are the highest ones
            bold_ref_too: If True, best values in X_ref are also in bold.
            bold_equal: If bold_equal is True, then the values equal to the 
            best ones are also bolded
            use_abs: If True, then the absolute values are used to compare
        Output:
            X: 1D numpy array with the best values in bold (or a tuple of two
            1D numpy arrays if bold_ref_too is True)
    """
    if use_abs:
        X_abs = np.abs(X)
        X_ref_abs = np.abs(X_ref)
    else:
        X_abs = X
        X_ref_abs = X_ref
    if higher_better:
        if bold_equal:
            best = np.greater_equal(X_abs, X_ref_abs)
        else:
            best = np.greater(X_abs, X_ref_abs)
    else:
        if bold_equal:
            best = np.less_equal(X_abs, X_ref_abs)
        else:
            best = np.less(X_abs, X_ref_abs)
    # Make bold
    X = np.array([f'\\textbf{{{x}}}' if best[i] else f'{x}' for i, x in enumerate(X_abs)])
    if bold_ref_too:
        X_ref = np.array([f'\\textbf{{{x}}}' if not best[i] else f'{x}' for i, x in enumerate(X_ref_abs)])
        return X, X_ref
    else:
        return X

In [ ]:
# Test
X = np.array([1, 2, 3, 4, 5])
X_ref = np.array([5, 4, 3, 2, 1])

res = bold_best(X, X_ref, higher_better=True, bold_ref_too=False)
test_eq(res, (np.array(['1', '2', '3', '\\textbf{4}', '\\textbf{5}'])))

res = bold_best(X, X_ref, higher_better=True, bold_ref_too=False, bold_equal=True)
test_eq(res, (np.array(['1', '2', '\\textbf{3}', '\\textbf{4}', '\\textbf{5}'])))

res = bold_best(X, X_ref, higher_better=False, bold_ref_too=False)
test_eq(res, (np.array(['\\textbf{1}', '\\textbf{2}', '3', '4', '5'])))

# TODO: Bold 3? 
res = bold_best(X, X_ref, higher_better=True, bold_ref_too=True)
test_eq(res, (np.array(['1', '2', '3', '\\textbf{4}', '\\textbf{5}']),
                np.array(['\\textbf{5}', '\\textbf{4}', '\\textbf{3}', '2', '1'])))

# Test use_abs
X = np.array([-1, 2, 3, 4, 5])
X_ref = np.array([-5, 4, 3, 2, 1])

res = bold_best(X, X_ref, higher_better=True, bold_ref_too=False, use_abs=True)
test_eq(res, (np.array(['1', '2', '3', '\\textbf{4}', '\\textbf{5}'])))
res = bold_best(X, X_ref, higher_better=True, bold_ref_too=False, use_abs=False)
test_eq(res, (np.array(['\\textbf{-1}', '2', '3', '\\textbf{4}', '\\textbf{5}'])))



In [ ]:
#|export
def convert_uuids_to_indices():
    cuda_visible_devices = os.environ.get("CUDA_VISIBLE_DEVICES", "")
    uuids = re.findall(r"\b[0-9a-fA-F]{8}(?:-[0-9a-fA-F]{4}){3}-[0-9a-fA-F]{12}\b", cuda_visible_devices)

    if uuids:
        indices = [str(i) for i in range(len(uuids))]
        os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(indices)

In [ ]:
#|export
def get_classified_columns (df: pd.DataFrame, thresholds:dict):
    """
    Creates classified columns based on predefined ranges for specified columns in the DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The DataFrame with classification classification of each column.

    """
    columns_to_classify = df.columns.intersection(thresholds.keys())
    solact_levels = ['low', 'moderate', 'elevated', 'high'] 

    if columns_to_classify.empty:
        return df
    else:
        df_cat = pd.DataFrame()
        for column in columns_to_classify:
            # ranges tuples come as strings in the yaml file, so we need to convert them to tuples with eval
            bins = pd.IntervalIndex.from_tuples(thresholds[column])
            df_cat[f'{column}_Cat'] = np.array(solact_levels)[pd.cut(df[column], bins=bins).cat.codes]
        return df_cat

In [ ]:
# Test 

data = {
    'F10': [50, 100, 160, 200],
    'S10': [30, 70, 170, 220],
    'M10': [60, 100, 150, 170],
    'Y10': [50, 90, 150, 170]
}
df = pd.DataFrame(data)


thresholds = {
    'F10': [(0,75), (76,150), (151,190), (191, df['F10'].max())],
    'S10': [(0,65), (66,150), (151,215), (216, df['S10'].max())],
    'M10': [(0,72), (73,144), (145,167), (168, df['M10'].max())],
    'Y10': [(0,81), (82,148), (149,165), (166, df['Y10'].max())]
}


# Expected result
expected_data = {
    'F10_Cat': ['low', 'moderate', 'elevated', 'high'],
    'S10_Cat': ['low', 'moderate', 'elevated', 'high'],
    'M10_Cat': ['low', 'moderate', 'elevated', 'high'],
    'Y10_Cat': ['low', 'moderate', 'elevated', 'high']
}
expected_df = pd.DataFrame(expected_data)

result_df = get_classified_columns(df, thresholds=thresholds)

print(result_df.head())

pd.testing.assert_frame_equal(result_df, expected_df)
print("Test passed!")


In [ ]:
#|export

def clean_f10_values(value):
    try:
        # Remove any trailing '+' using regex and convert to float
        cleaned_value = re.sub(r'\+$', '', value)
        return float(cleaned_value)
    except ValueError:
        # If conversion fails, return NaN
        return np.nan

def get_F10_historical_distribution(thresholds:dict):  
  """
  Calculate the distribution of F10.7 values from historical data.

  Returns:
  dict: A dictionary containing the normalized value counts of F10.7 values and its categories.
  """

  data = config.data

  # From here we will extract the data of F10.7 to check its distribution
  fnameF107_historical = data.dataF107_path if data.dataF107_url is None else download_data(data.dataF107_url,
                                                                              fname=data.dataF107_path)

  # Only two columns in the file: Date and F10.7.
  df_F10 = pd.read_csv(
            fnameF107_historical, delim_whitespace=True, comment='#', header=None, 
            names=['Date', 'F10'], 
            parse_dates=['Date'], 
            na_values=[".", "+"], 
            converters={'F10': clean_f10_values},
          )  
  df_F10['F10'] = df_F10['F10'].astype(float)

  # Data preprocessing 
  # Remove the first rows with missing values
  df_F10 = df_F10[((df_F10['Date'] >= '1947-02-14') & (df_F10['Date'] < '1997-01-01'))]

  # Fill the missing values with the average of the previous and next value
  df_F10['F10'] = ((df_F10['F10'].fillna(method='ffill')) + df_F10['F10'].fillna(method='bfill'))/2

  df_F107_cat = pd.Series(get_classified_columns(df_F10, thresholds)['F10_Cat'])
  value = df_F107_cat.value_counts(normalize=True).to_dict()
  
  return value

In [ ]:
# Test
get_F10_historical_distribution(thresholds)  

In [ ]:
#|export
def euclidean_distance_dict(X:dict, Y:dict):
    return math.sqrt(sum((X.get(d,0) - Y.get(d,0))**2 for d in set(X) | set(Y)))


In [ ]:
# Test 

# Test case 1: Basic test with non-overlapping keys
X1 = {'a': 1, 'b': 2, 'c': 3}
Y1 = {'d': 4, 'e': 5, 'f': 6}
expected_distance1 = np.sqrt(1**2 + 2**2 + 3**2 + 4**2 + 5**2 + 6**2)
assert np.isclose(euclidean_distance_dict(X1, Y1), expected_distance1), f"Test case 1 failed"

# Test case 2: Basic test with overlapping keys
X2 = {'a': 1, 'b': 2, 'c': 3}
Y2 = {'a': 1, 'b': 2, 'c': 4}
expected_distance2 = np.sqrt(0**2 + 0**2 + 1**2)
assert np.isclose(euclidean_distance_dict(X2, Y2), expected_distance2), f"Test case 2 failed"

# Test case 3: Basic test with some overlapping and some non-overlapping keys
X3 = {'a': 1, 'b': 2}
Y3 = {'b': 2, 'c': 3}
expected_distance3 = np.sqrt(1**2 + 0**2 + 3**2)
assert np.isclose(euclidean_distance_dict(X3, Y3), expected_distance3), f"Test case 3 failed"

# Test case 4: Test with empty dictionaries
X4 = {}
Y4 = {}
expected_distance4 = 0
assert np.isclose(euclidean_distance_dict(X4, Y4), expected_distance4), f"Test case 4 failed"

# Test case 5: Test with one empty dictionary
X5 = {'a': 1, 'b': 2}
Y5 = {}
expected_distance5 = np.sqrt(1**2 + 2**2)
assert np.isclose(euclidean_distance_dict(X5, Y5), expected_distance5), f"Test case 5 failed"

print("All test cases passed!")

In [ ]:
#|export

def find_closest_distribution(df_cat, target_distribution, segment_size, val_size):
    """
    Finds the combination of segments in the categorical data that is closest to the target distribution.

    Parameters:
    df_cat (pd.Series): A pandas Series containing the categorical data.
    target_distribution (dict): The target distribution to compare against, given as a dictionary where keys are categories and values are their target proportions.
    segment_size (int): The size of each segment to split the data into.
    val_size (float): The proportion of the validation split.

    Returns:
    best_combination (tuple): The indices of the segments that form the closest combination to the target distribution.
    segments (list): The list of segments created from the data.
    distribution_found (dict): The distribution of categories in the best combination of segments.
    """
    idxs = list(df_cat.index)
    segments = np.array_split(idxs, len(df_cat) // segment_size)

    value_counts = [df_cat[segments[i]].value_counts().to_dict() for i in range(len(segments))]

    num_segments = int(len(segments)*(val_size))
    print(f"Total number of segments:{ len(segments)}, Number of segments for validation: {num_segments} ({num_segments/len(segments)*100:.2f}%)")

    
    best_combination = None
    best_distance = np.inf
    distribution_found = None
    comb = combinations(range(len(value_counts)), num_segments)
    for c in tqdm(comb):
        values = Counter({})
        for i in c:
            values = values + Counter(value_counts[i])
        total = sum(values.values(), 0.0)
        distribution = {k: v / total for k, v in values.items()}
        
        distance = euclidean_distance_dict(distribution, target_distribution)

        if distance < best_distance:
            best_distance = distance
            best_combination = c
            distribution_found = distribution
    print("The closest group of segments to F10.7 categories has an euclidean distance of", best_distance)
    return best_combination, segments, distribution_found

In [ ]:
# Test

target_distribution = {'A': 0.25, 'B': 0.25, 'C': 0.25, 'D': 0.25}

data = {
    'category': ['A', 'B', 'A', 'C', 'D', 'B', 'A', 'C', 'D', 'B', 'A', 'C', 'D', 'B', 'A', 'C', 'D', 'B', 'A', 'C']
}
df_cat = pd.Series(data['category'])

# Function parameters
segment_size = 5
val_size = 0.4

best_combination, segments, distribution_found = find_closest_distribution(df_cat, target_distribution, segment_size, val_size)

print("Best combination of segments:", list(best_combination))
print("Distribution found:", distribution_found)

In [ ]:
#|eval: false
#|hide
from nbdev import *
nbdev_export()